In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@151ca10f


Some(http://iuni2:4040)

In [3]:
//val sqlContext = new SQLContext(sc)
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("/WoSraw/WoS2019/xmlRaw/*.xml.gz")

df.show(2)
df.printSchema()

+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+----+--------+---------+-----------------+--------------------+----+
|                UID|    _r_id_disclaimer|abstracts|addresses|category_info|citedWork|contributors|        dynamic_data|fund_ack|   i|item|keywords|reference|reprint_addresses|         static_data| sub|
+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+----+--------+---------+-----------------+--------------------+----+
|WOS:000205001600028|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|null|    null|     null|             null|[,,,,, [, [, 1, [...|null|
|WOS:000202412000001|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|null|    null|     null|             null|[,,,,, [, [,

df = [UID: string, _r_id_disclaimer: string ... 14 more fields]


[UID: string, _r_id_disclaimer: string ... 14 more fields]

In [4]:
import org.apache.spark.sql.SaveMode
df.write.mode(SaveMode.Overwrite)
.parquet("/WoSraw/WoS2019/parquet")
//WoS2.describe().filter($"summary" === "count").show

In [17]:
val WoS = spark.read.format("parquet").load("/WoSraw/WoS2019/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name", "static_data.fullrecord_metadata.reprint_addresses",
                      "static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword","static_data.fullrecord_metadata.references.reference",
                      "static_data.contributors.contributor","static_data.summary.EWUID",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 14 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 14 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 14 more fields]

In [64]:
WoS2.write.mode(SaveMode.Overwrite).json("/WoSraw/json")

In [110]:
val WoS3 = WoS2.select("UID", "pub_info._pubtype", "pub_info._pubyear", "fund_ack.fund_text", "titles", 
                       "identifier", "name","publishers.publisher.names.name.full_name","address_name.address_spec.full_address")
val WoSIU = WoS3.withColumn("allAddresses", array_join($"full_address", "|")).filter($"allAddresses".contains("Indiana Univ")).cache()
WoSIU.describe().filter($"summary" === "count").show

+-------+------+--------+--------+---------+------------+
|summary|   UID|_pubtype|_pubyear|full_name|allAddresses|
+-------+------+--------+--------+---------+------------+
|  count|133767|  133767|  133767|   133766|      133767|
+-------+------+--------+--------+---------+------------+



WoS3 = [UID: string, _pubtype: string ... 7 more fields]
WoSIU = [UID: string, _pubtype: string ... 8 more fields]


[UID: string, _pubtype: string ... 8 more fields]

In [111]:
val WoSIU2 = WoSIU.filter(($"_pubyear"===2018||$"_pubyear"===2019))//&&$"full_name".contains("ELSEVIER"))
//WoSIU2.describe().filter($"summary" === "count").show
val WoSIU3 = WoSIU2.withColumn("FX", array_join($"fund_text.p", "."))
WoSIU3.show

+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                UID|_pubtype|_pubyear|           fund_text|              titles|          identifier|                name|           full_name|        full_address|        allAddresses|                  FX|
+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|WOS:000418232300001| Journal|    2018|[[We thank Drs. K...|[6, [[HUMAN GENE ...|[[, accession_no,...|[[1 2,,,, author,...|MARY ANN LIEBERT,...|[Indiana Univ Sch...|Indiana Univ Sch ...|We thank Drs. Ken...|
|WOS:000414522900001| Journal|    2018|[[This work was s...|[6, [[JOURNAL OF ...|[[, accession_no,...|[[1,,, Y, author,...|MARY ANN LIEBERT,...|[Ohio State Univ,...|Ohi

WoSIU2 = [UID: string, _pubtype: string ... 8 more fields]
WoSIU3 = [UID: string, _pubtype: string ... 9 more fields]


[UID: string, _pubtype: string ... 9 more fields]

In [112]:
import scala.util.Try
import org.apache.hadoop.fs._
import org.apache.spark.sql.Row

def getTitle(_type: String) = { 
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, _, s) => s == _type}
      .map(_._1)
      .orElse(None)
  )
}
val WoSIU4 = WoSIU3.withColumn("journal", when(col("titles.title").isNotNull, getTitle("source")($"titles.title")).otherwise(lit(null))).cache()
WoSIU4.show()

+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                UID|_pubtype|_pubyear|           fund_text|              titles|          identifier|                name|           full_name|        full_address|        allAddresses|                  FX|             journal|
+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|WOS:000418232300001| Journal|    2018|[[We thank Drs. K...|[6, [[HUMAN GENE ...|[[, accession_no,...|[[1 2,,,, author,...|MARY ANN LIEBERT,...|[Indiana Univ Sch...|Indiana Univ Sch ...|We thank Drs. Ken...|HUMAN GENE THERAP...|
|WOS:000414522900001| Journal|    2018|[[This work was s...|[6, [[JOURNAL OF ...|[[,

WoSIU4 = [UID: string, _pubtype: string ... 10 more fields]


getTitle: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, _pubtype: string ... 10 more fields]

In [113]:
//WoSIU4.write.mode(SaveMode.Overwrite).parquet("/WoSraw/WoS2019/IUpapersExtract")
//val WoSIU4b = spark.read.format("parquet").load("/WoSraw/WoS2019/IUpapersExtract/*.parquet")
val WoSIU5 = WoSIU4.withColumn("title", when(col("titles.title").isNotNull, getTitle("item")($"titles.title")).otherwise(lit(null)))
WoSIU5.show

+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                UID|_pubtype|_pubyear|           fund_text|              titles|          identifier|                name|           full_name|        full_address|        allAddresses|                  FX|             journal|               title|
+-------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|WOS:000418232300001| Journal|    2018|[[We thank Drs. K...|[6, [[HUMAN GENE ...|[[, accession_no,...|[[1 2,,,, author,...|MARY ANN LIEBERT,...|[Indiana Univ Sch...|Indiana Univ Sch ...|We thank Drs. Ken...|HUMAN GENE THERAP...|Improving the Tra...|


WoSIU5 = [UID: string, _pubtype: string ... 11 more fields]


[UID: string, _pubtype: string ... 11 more fields]

In [114]:
def getID(_type: String) = {
  udf((row: Seq[Row]) =>
    row.map(r => (r.getString(0), r.getString(1), r.getString(2)))
      .find { case (_, s, _) => s == _type}
      .map(_._3)
      .orElse(None)
  )
}


val WoSIU6 = WoSIU5.withColumn("doi", when(col("identifier").isNotNull, getID("doi")($"identifier")).otherwise(lit(null)))
.withColumn("issn", when(col("identifier").isNotNull, getID("issn")($"identifier")).otherwise(lit(null)))
.withColumn("eissn", when(col("identifier").isNotNull, getID("eissn")($"identifier")).otherwise(lit(null)))
//.select("UID","doi","issn","eissn").filter($"doi" =!= "null").show
WoSIU6.describe().filter($"summary" === "count").show

+-------+-----+--------+--------+---------+------------+----+-------+-----+-----+-----+-----+
|summary|  UID|_pubtype|_pubyear|full_name|allAddresses|  FX|journal|title|  doi| issn|eissn|
+-------+-----+--------+--------+---------+------------+----+-------+-----+-----+-----+-----+
|  count|16464|   16464|   16464|    16464|       16464|8413|  16464|16464|13233|15797|13375|
+-------+-----+--------+--------+---------+------------+----+-------+-----+-----+-----+-----+



WoSIU6 = [UID: string, _pubtype: string ... 14 more fields]


getID: (_type: String)org.apache.spark.sql.expressions.UserDefinedFunction


[UID: string, _pubtype: string ... 14 more fields]

In [141]:
val WoSIU7 = WoSIU6.withColumn("reprintFlag", array_position($"name._reprint", "Y"))
            .select("UID","_pubtype","_pubyear","eissn","issn","doi","title","journal","full_name","FX","reprintFlag","name.display_name","allAddresses")
            .toDF("WoSid","PT","PY","EI","SN","DI","TI","SO","PU","FX","RP","authors","C1")
WoSIU7.show(false)

+-------------------+-------+----+---------+---------+----------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------+-----------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

|WOS:000414274200002|Journal|2018|1931-843X|1540-9996|10.1089/jwh.2017.6439             |Paternal Acculturation and Maternal Health Behaviors: Influence of Father's Ethnicity and Place of Birth                                                                                                  |JOURNAL OF WOMENS HEALTH                                         |MARY ANN LIEBERT, INC        |null                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

|WOS:000439102000002|Journal|2018|1526-632X|0028-3878|10.1212/WNL.0000000000005213      |Attention-deficit/hyperactivity disorder medication and seizures                                                                                                                                          |NEUROLOGY                                                        |LIPPINCOTT WILLIAMS & WILKINS|This project was supported by NIMH grant R01MH102221, NIH grant T32HD07475, and NIDA grant K99DA040727. MarketScan is a registered trademark of Truven Health Analytics Inc.                                                                                                                                                                                                                                                                                                                                                                                                                                                        

|WOS:000428505700001|Journal|2018|2470-0029|2470-0010|10.1103/PhysRevD.97.052011        |Improved measurements of X-cJ -> Sigma(+) (Sigma)over-bar(-) and Sigma(0)(Sigma)over-bar(0) decays                                                                                                        |PHYSICAL REVIEW D                                                |AMER PHYSICAL SOC            |The BESIII Collaboration thanks the staff of BEPCII and the IHEP computing center for their strong support. This work is supported in part by National Key Basic Research Program of China under Contract No. 2015CB856700; National Natural Science Foundation of China (NSFC) under Contracts No. 11375204, No. 11505034, No. 11235011, No. 11335008, No. 11425524, No. 11625523, No. 11635010; the Chinese Academy of Sciences (CAS) Large-Scale Scientific Facility Program; the CAS Center for Excellence in Particle Physics (CCEPP); the Collaborative Innovation Center for Particles and Interactions (CICPI); Joint Large-

|WOS:000449567500007|Journal|2018|1740-634X|0893-133X|10.1038/s41386-018-0166-x         |Striatal activity correlates with stimulant-like effects of alcohol in healthy volunteers                                                                                                                 |NEUROPSYCHOPHARMACOLOGY                                          |NATURE PUBLISHING GROUP      |This research was supported by the National Institute on Alcohol Abuse and Alcoholism Grant R21 AA017502 (E.C.). J.W. was supported by National Institute on Alcohol Abuse and Alcoholism Grant K01 AA024519. T.J.R. and E.A.S. were supported by the National Institute on Drug Abuse Intramural Research Program. S.O. was supported by National Institute on Alcohol Abuse and Alcoholism grant P60 AA007611. The funding agencies had no involvement in the research other than financial support.                                                                                                                              

WoSIU7 = [WoSid: string, PT: string ... 11 more fields]


[WoSid: string, PT: string ... 11 more fields]

In [146]:
//val WoSIU8 = WoSIU7.filter($"PU".contains("ELSEVIER"))
val WoSIU8 = WoSIU7.withColumn("AU", array_join($"authors", "|"))
//WoSIU8.describe().filter($"summary" === "count").show
WoSIU8.select("WoSid","PT","PY","EI","SN","DI","TI","SO","PU","FX","RP","AU","C1")
    .coalesce(1).write.mode(SaveMode.Overwrite).format("com.databricks.spark.csv")
    .option("header", "true")
    .option("codec", "org.apache.hadoop.io.compress.GzipCodec")
    .save("/WoSraw/WoS2019/IUpapersExtract")

WoSIU8 = [WoSid: string, PT: string ... 12 more fields]


[WoSid: string, PT: string ... 12 more fields]